In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
import pickle
import tqdm as tqdm

In [49]:

max_words = 20000  # Maximum number of unique words to consider
max_sequence_length = 250  # Maximum length of each
embedding_dim = 300  # Dimension of the GloVe word embeddings
negation_words = ['not', 'no', 'never']  # Words to be considered as negation words
accumulation_steps = 4
learning_rates = [2e-5, 5e-5, 1e-4]
patience = 2  # Set the number of epochs to wait before stopping if validation loss doesn't decrease

def sentiment_to_binary(sentiment):
    if sentiment == 'positive':
        return 1
    else:
        return 0


def preprocess_text_with_negation(text):
    # Convert the text to lowercase
    text = text.lower()

    # for negation_word in negation_words:
    #     if negation_word in text:
    #         text = text.replace(negation_word, 'not_')

    # Remove any non-alphanumeric characters
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    
    return text

In [50]:
df1 = pd.read_csv('../IMDB Dataset.csv')

df1.head()

,review,sentiment
0,not good moive,negative
1,I absolutely loved this movie. The plot was engaging and the characters were incredibly well-developed.,positive
2,What a disappointment. The storyline was weak and the acting was subpar. I would not recommend this film.,negative
3,"The cinematography in this film was breathtaking, and the acting was top-notch. A must-see!",positive
4,"I struggled to stay awake during this movie. The pacing was slow, and the dialogue was painful to listen to.",negative


In [51]:
df2 = pd.read_csv('../Reviews.csv')
df2.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [52]:
df2 = df2[['Text', 'Score']]
df2 = df2.rename(columns={'Text': 'review', 'Score': 'sentiment'})
df2['sentiment'] = df2['sentiment'].apply(lambda x: 'positive' if x > 3 else 'negative')

In [54]:
df = pd.concat([df1, df2], ignore_index=True)


In [ ]:
# Apply the preprocess_text_with_negation function to the 'review' column
df['review'] = df['review'].apply(preprocess_text_with_negation)

df['sentiment'] = df['sentiment'].apply(sentiment_to_binary)


In [61]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [62]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
# Tokenize the text
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=250)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=250)

In [64]:
# Convert the tokenized data into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(1000).batch(4)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(4)


In [65]:
import tensorflow as tf
from transformers import TFBertModel
from keras.layers import Dense, Dropout

class CustomBertModel(tf.keras.Model):
    def __init__(self, config, num_classes):
        super(CustomBertModel, self).__init__()
        
        self.bert = TFBertModel.from_pretrained('bert-base-uncased', config=config)
        self.dropout = Dropout(config.hidden_dropout_prob)
        self.dense1 = Dense(256, activation='relu')
        self.dense2 = Dense(num_classes, activation='linear')

    def call(self, inputs, **kwargs):
        outputs = self.bert(inputs, **kwargs)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.dense1(pooled_output)
        logits = self.dense2(pooled_output)
        return logits


In [ ]:
# Fine-tune the model
#Going to take 5 hours to train each epoch
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

print('Training the model...')
model.fit(train_dataset, epochs=3, batch_size=4, validation_data=test_dataset)
# Evaluate the model
y_pred_logits = model.predict(test_dataset, batch_size=4)
y_pred = np.argmax(y_pred_logits.logits, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))
# Save the fine-tuned model
model.save_pretrained('sentiment_analysis_bert_custome_model/')
tokenizer.save_pretrained('sentiment_analysis_bert_custome_model/')


In [3]:
#Predicting Sentiment with a Fine-Tuned BERT Model
# Load the pre-trained model and tokenizer
model_path = 'sentiment_analysis_bert_'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = TFBertForSequenceClassification.from_pretrained(model_path)

def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)

    # Perform the sentiment prediction
    outputs = model(inputs)
    logits = outputs.logits

    # Convert the logits to probabilities
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()

    # Get the predicted class (0: Negative, 1: Positive)
    predicted_class = np.argmax(probabilities, axis=-1)

    return predicted_class, probabilities

# while loop to take input from user
while True:
    # Take input from user
    text = input("Enter a sentence to predict its sentiment (or 'q' to quit): ")

    text = preprocess_text_with_negation(text)    
    # Check if the user wants to quit
    if text == 'q':
        break
    
    # Predict the sentiment
    predicted_class, probabilities = predict_sentiment(text)
    
    # Print the results
    print(text)
    print("Predicted sentiment: {}".format(predicted_class[0]))
    print("Negative probability: {}".format(probabilities[0][0]))
    print("Positive probability: {}".format(probabilities[0][1]))


NameError: name 'BertTokenizer' is not defined

In [1]:
# Predict the sentiment without a fine-tuned model
# while loop to take input from user
while True:
    # Take input from user
    text = input("Enter a sentence to predict its sentiment (or 'q' to quit): ")

    text = preprocess_text_with_negation(text)    
    # Check if the user wants to quit
    if text == 'q':
        break
    
    # Predict the sentiment
    predicted_class, probabilities = predict_sentiment(text)
    
    # Print the results
    print(text)
    print("Predicted sentiment: {}".format(predicted_class[0]))
    print("Negative probability: {}".format(probabilities[0][0]))
    print("Positive probability: {}".format(probabilities[0][1]))


NameError: name 'preprocess_text_with_negation' is not defined

In [21]:
from transformers import pipeline

# Load the sentiment analysis pipeline
nlp = pipeline("sentiment-analysis")

# Loop to take user input and get the sentiment analysis output
while True:
    user_input = input("Enter a sentence: ")
    if user_input.lower() == "quit":
        break
    else:
        result = nlp(user_input)
        print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.2f}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are 

Sentiment: POSITIVE, Score: 1.00
Sentiment: NEGATIVE, Score: 1.00
Sentiment: POSITIVE, Score: 1.00
Sentiment: POSITIVE, Score: 1.00
Sentiment: NEGATIVE, Score: 1.00
Sentiment: NEGATIVE, Score: 0.94


In [10]:
# Continue to train the model 
model_path = 'sentiment_analysis_bert_'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = TFBertForSequenceClassification.from_pretrained(model_path)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(train_dataset, epochs=3, batch_size=4, validation_data=test_dataset)

# Save the fine-tuned model
model.save_pretrained('sentiment_analysis_bert_continue/')

Some layers from the model checkpoint at sentiment_analysis_bert_ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at sentiment_analysis_bert_.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Epoch 1/3
 2634/10017 [======>.......................] - ETA: 21:30 - loss: 0.0607 - accuracy: 0.9799